# Analiza danych przestępst popełnianych w Lost Angeles w latach 2020 - 2023
W projekcie zdecydowaliśmy się wykorzystać dwa kontenery: jeden zawiera bazę danych **MSSQL**, a drugi - Sparka z zainstalowanym **Pythonem**, umożliwiającym komunikację za pomocą biblioteki **Pyspark**. Spark w kontenerze efektywnie współpracuje z bazą MSSQL, umożliwiając przetwarzanie danych bezpośrednio na bazie, co różni się od podejścia popularnego pakietu **Pandas**. To rozwiązanie pozwala na wydajne przetwarzanie dużych zbiorów danych bez potrzeby instalowania dodatkowych zależności na lokalnej maszynie.

### Utwoprzenie nowej bazy danych

Poniższy kod automatyzuje proces tworzenia nowej bazy danych w systemie zarządzania bazami danych **MSSQL**. Wykorzystuje do tego bibliotekę **pyodbc**, aby ułatwić nawiązanie połączenia z bazą danych za pomocą określonych danych uwierzytelniających i sterownika **ODBC**. Następnie, po ustanowieniu połączenia, kod wykonuje polecenie SQL `CREATE DATABASE`, aby utworzyć nową bazę danych o nazwie "AFTER_ETL_CRIME_LA". Po zakończeniu tego procesu, zamyka połączenie z bazą danych. Na koniec, wyświetla komunikat potwierdzający poprawne utworzenie nowej bazy danych.

<img src="Tworzenie_bazy.png">
<br>
<p style="text-align:center"><i>Schemat przedstawiający sposób tworzenia bazy za pomocą biblioteki pyodbc</i></p>

```python
import pyodbc

# Parametry połączenia
server = 'localhost'
database = 'master'  # Połącz się z bazą master, aby móc utworzyć nową bazę danych
username = 'sa'
password = 'YourStrongPassword123'
driver = '{ODBC Driver 17 for SQL Server}'

# Tworzenie ciągu połączenia
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Nawiązanie połączenia
conn = pyodbc.connect(conn_str)
conn.autocommit = True
# Utworzenie kursora
cursor = conn.cursor()
# Polecenie SQL CREATE DATABASE
create_db_query = "CREATE DATABASE AFTER_ETL_CRIME_LA"

# Wykonanie polecenia
cursor.execute(create_db_query)

# Zatwierdzenie zmian
conn.commit()

# Zamykanie kursora i połączenia
cursor.close()
conn.close()

print("Baza danych Clean_Database_1 została utworzona.")

### Tworzenie nowej ramki danych

Ten skrypt Pythona używa biblioteki Pandas do wczytania danych z pliku CSV i następnie przesyła te dane do bazy danych SQL Server. Najpierw określa parametry połączenia, takie jak nazwa serwera, nazwa bazy danych, użytkownik i hasło. Następnie tworzy silnik SQLAlchemy, który ułatwia komunikację z bazą danych. Dane są wczytywane z pliku CSV i przetwarzane, aby wybrać co dziesiąty wiersz. Na koniec dane są dodawane do tabeli 'Pracownicy15' w bazie danych. Po zakończeniu procesu, wyświetlany jest komunikat potwierdzający sukces operacji.

<img src="Tabela.png" width="800" height="400">
<br>
<p style="text-align:center"><i>Schemat tworzenia nowej tabeli oraz</i></p>

```python
import pandas as pd
from sqlalchemy import create_engine

# Parametry połączenia
server = 'localhost'
database = 'NowaBazaDanych2'
username = 'SA'
password = 'YourStrongPassword123'
driver = 'ODBC Driver 17 for SQL Server'

# Tworzenie silnika SQLAlchemy
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}')

# Ładowanie danych z pliku CSV
df = pd.read_csv('dane.csv', encoding='utf-8')

# Wybierz tylko co 1000 wiersz
df = df.iloc[::10]

# Mapowanie DataFrame na istniejącą tabelę w bazie danych
df.to_sql('Pracownicy15', engine, if_exists='append', index=False)

print("Wybrane dane zostały dodane do tabeli w bazie danych.")